**ASSIGNMENT 1 - EMPIRICAL STUDY OF KNAPSACK PROBLEM**

**1. Group Description**

Group Number: 61

Member Names: Owen Jory

Member Student Numbers: 300168367

**2. Knapsack Problem**

Give a description of the problem tackled.

**3. Dataset**

Give a description of the dataset used with references.  

**Import important libraries**

In [1]:
import pandas as pd
import itertools
import numpy as np

**Read Dataset**

As outlined in the project description, it should be possible for the correctors to execute your notebook without requiring any downloads.

To facilitate access to the dataset without the need for downloads, you can upload it to a public GitHub repository and provide a link to the raw version of the dataset.

The link to the raw version is as follows:
*https://raw.githubusercontent.com/GITHUB_USERNAME/REPOSITORY_NAME/main/DATASETNAME.csv*

For example:

https://raw.githubusercontent.com/baharin/KnapsackProblem/main/knapsack_5_items.csv

Now provide the link to YOUR dataset and read the dataset using pandas:

In [2]:
# personally, I prefer to avoid unnecessarily tethering my programs to the internet, so I'll just link a file rather than github. I hope thats okay :)
url = "https://raw.githubusercontent.com/SmolTortol/KnapsackProblem/main/knapsack_5_items.csv?token=GHSAT0AAAAAACIIOJSJ3PHCLHXWS45BDOQCZIYGQJA"

dataset = pd.read_csv(url)

Let's see what are the columns of the dataset? :

In [3]:
dataset.columns

Index(['Weights', 'Prices', 'Capacity', 'Best picks', 'Best price'], dtype='object')

As we expected, we have columns for weights, costs, capacity, best picks and best price for all the instances.

Now let's see the first 10 entries (rows):

In [4]:
dataset.head(10)

,Weights,Prices,Capacity,Best picks,Best price
0,[46 40 42 38 10],[12 19 19 15 8],40,[0. 1. 0. 0. 0.],19.0
1,[11 31 4 6 7],[ 2 8 18 16 3],64,[1. 1. 1. 1. 1.],47.0
2,[32 49 27 37 24],[19 16 16 4 1],87,[1. 0. 1. 0. 1.],36.0
3,[20 35 22 23 16],[19 17 19 9 1],21,[1. 0. 0. 0. 0.],19.0
4,[ 7 12 19 13 20],[10 11 18 15 5],50,[0. 1. 1. 1. 0.],44.0
5,[27 10 25 25 7],[13 19 7 16 3],66,[1. 1. 0. 1. 0.],48.0
6,[21 2 33 45 26],[ 1 14 10 6 13],80,[0. 1. 1. 0. 1.],37.0
7,[37 27 39 14 25],[18 7 15 4 13],35,[0. 0. 0. 0. 1.],13.0
8,[ 1 48 4 23 39],[ 9 4 10 16 12],51,[1. 0. 1. 1. 0.],35.0
9,[ 4 3 22 9 32],[14 6 3 17 8],53,[1. 1. 0. 1. 1.],45.0


**Preprocessing Step**

Typically, the initial step in any project that involves reading and handling data is data preprocessing and cleansing.

In our dataset, we expect the entries in the "Weights," "Prices," and "Best Picks" columns to be in the form of arrays of floats or integers, like this: [45, 40, 42, 38, 10]

However, when you read each entry using pandas, they will be in a form of a string: "[45 40 42 38 10]"

So we need to convert these strings into "arrays of floats or integers." You can utilize the function provided below for this purpose:


In [5]:
def string_to_list(string):

  string_list = string.strip('[]').split()

  float_list = [float(element) for element in string_list]

  return float_list

Furthermore, it's possible that certain rows in the dataset contain empty values in specific columns. We also aim to eliminate these rows as they do not provide any useful information. We use dropna() function to do so:

In [6]:
#Ignore the warning messages.

dataset = dataset.dropna()

dataset.Weights = dataset.Weights.apply(lambda x : string_to_list(x))
dataset.Prices = dataset.Prices.apply(lambda x : string_to_list(x))
dataset['Best picks'] = dataset['Best picks'].apply(lambda x : string_to_list(x))

Now it's time to implement the search algorithms. For each algorithm, a template is provided to you. You can modify this template if you want. But first you should try to go look at all the parameters used, as they are all important. You can also define any number of auxiliary functions you want.


**4. Generate and Test**

Just check everything. That'll do it!

In [7]:
def gen_and_test(data):
    # Dummy values, because the dummy solution hast -inf cost, it will be replaced as soon as any real data is processed
    best_solution_price = 0
    best_solution = None 

    # I'm using a binary counter, because I've been playing waaaaay too much Minecraft lately
    # We'll start by giving each solution an "id"
    for solution_id in range(2**len(data['Weights'])):
        # Knapsacs are typically empty when there is nothing in them, hence, 0 price and 0 weight
        solution_price = 0
        solution_weight = 0
        solution = [-1]*len(data['Weights'])

        # Convert the solution "id" into its binary representation
        for i in range(len(data['Weights'])):
            solution[i] = solution_id // (2**i) % 2
            
            # Increase the price and weight if there is a 1 for the current bit
            solution_price += solution[i]*data['Prices'][i]
            solution_weight += solution[i]*data['Weights'][i]

        # If this solution is better that the best we've seen so far (or if we haven't seen any yet), this is now the best solution we've seen
        if solution_weight <= data['Capacity'] and solution_price > best_solution_price:
            best_solution_price = solution_price
            best_solution = solution

    # There is a faster solution to the {0, 1}-knapsack problem using dynamic programming, but, I don't think such a solution is in the nature of the assignment 
    return best_solution_price, best_solution



In [8]:
solutions = []
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = gen_and_test(row)
    solutions.append(1 if target == solution else 0)


In [9]:
# Accuracy
print('Accuracy of best prices found is', np.mean(solutions))

Accuracy of best prices found is 1.0


**Your Analysis:**

Generate and test is extremely accurate. Unfortunatly, my algorithm here is extremely slow, as are the optimal generate and test algorithms for many problems 

------------------------------------------------------------------------------------------------

**5. Greedy Search**

Always take the most valuable items first

In [10]:
def greedy(data):
    # We're only going to generate one solution here
    best_solution_price = 0
    best_solution_weight = 0
    best_solution = [0]*len(data['Weights'])

    # Lets be greedy for prices, because we can
    shiny = 0
    while shiny is not None:
        # Before we look at any items, the best item we've seen is nothing
        shiny = None
        for i in range(len(data['Weights'])):
            # If an item fits in our knapsack, isn't in our knapsack, and is better than we've seen so far, it is now the best we've seen this far
            if data['Weights'][i] + best_solution_weight <= data['Capacity'] and best_solution[i] == 0 and ((shiny is None and data['Prices'][i] > 0) or data['Prices'][i] > data['Prices'][shiny]):
                shiny = i

        # Add the item to the knapsack
        if shiny is not None:
            best_solution_price += data['Prices'][shiny]
            best_solution_weight += data['Weights'][shiny]
            best_solution[shiny] = 1
            
    return best_solution_price, best_solution


In [11]:
solutions_greedy = []
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = greedy(row)
    solutions_greedy.append(1 if target == solution else 0)


In [12]:
print("Greedy Accuracy is", np.mean(solutions_greedy))

Greedy Accuracy is 0.9066707156594797


**Your Analysis:**

The greedy algorithm is very fast at generating a solution. However, this comes at the cost of accuracy. I though it was still shockingly accurate given how easy it is to confuse

------------------------------------------------------------------------------------------------

**6. Simulated Annealing**

Check a few other solutions before blindly accepting greed as law

In [13]:
# I didn't see these until afer i wrote the algorithm, whoops!
import random
import math


def simulated_annealing(data, N, initial_temperature, cooling_rate):
    # The initial candidate solution will be the greedy solution from before
    best_solution_price, best_solution = greedy(data)
    temperature = initial_temperature

    for _ in range(N):
        # generate a new solution by copying the old one and flipping a random bit 3 times (3 works better than 1 zero and 1 one for some reason)
        solution = best_solution[:]
        for i in range(3):
            mutation = np.random.randint(len(solution))
            solution[mutation] = 1 - solution[mutation]
        solution_price = calculate_energy(solution, data)

        if accept(best_solution_price-solution_price, temperature):
            best_solution_price, best_solution = solution_price, solution

        temperature *= cooling_rate

    return best_solution_price, best_solution


def accept(delta_E, T):
    if delta_E < 0:
        # if better, keep change
        return True
    else:
        # if worse, sometimes keep change
        if np.random.rand() < np.e**(-delta_E/T):
            return True
        else:
            return False


def calculate_energy(solution, data):
    solution_price = 0
    solution_weight = 0
    
    for i in range(len(solution)):
        # Increase the price and weight if there is a 1 for the current bit
        solution_price += solution[i]*data['Prices'][i]
        solution_weight += solution[i]*data['Weights'][i]
        
    if solution_weight <= data['Capacity']:
        return solution_price
    else:
        return 0

In [14]:
solutions_sa = []
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = simulated_annealing(row, N = 10, initial_temperature=1, cooling_rate=0.95)
    solutions_sa.append(1 if target == solution else 0)


In [15]:
print("Simulated Annealing Accuracy is", np.mean(solutions_sa))

Simulated Annealing Accuracy is 0.9157809494888146


**Your Analysis:**

Simulated annealing, or at least my implementaion of it here, is *slightly* better than a pure greedy search, however, than comes at the cost of doing a lot of redundant work. 

------------------------------------------------------------------------------------------------

**7. Genetic Algorithm**

Give a description... Also, make sure that you put comments using your own words in the code to show that you understand the code that you are submitting.

In [16]:
import random

def calculate_fitness(ind, prices, weights, capacity):
    price = 0
    weight = 0

    # Tally the weight and price
    for i in range(len(ind)):
        price += ind[i]*prices[i]
        weight += ind[i]*weights[i]

    # Punish overdraft
    fitness = price
    if weight > capacity:
        fitness = 0
    
    return fitness

def crossover(parent1, parent2, cross_rate):
    # start with a random parent
    parent = random.randrange(2)
    child = [-1]*len(parent1)

    for i in range(len(child)):
        # copy the current bit from the active parent
        if parent%2 == 0:
            child[i] = parent1[i]
        else:
            child[i] = parent2[i]

        # change the active parent with probability cross_rate
        if random.random() < cross_rate:
            parent += 1

    return child

def mutation(child, mut_rate):
    # flip bits with probability mut_rate
    for i in range(len(child)):
        if random.random() < mut_rate:
            child[i] = 1-child[i]

    return child


def pick_best(bracket, prices, weights, capacity):
    # No best if no check
    best = None
    best_price = float("-inf")

    # Grab the best
    for gladiator in bracket:
        if calculate_fitness(gladiator, prices, weights, capacity) > best_price:
            best = gladiator
            best_price = calculate_fitness(gladiator, prices, weights, capacity)

    return best_price, best


def genetic_algorithm(data, population_size, num_generations, mut_rate, cross_rate, tournament_size):   
    population = []

    # Populate the gene pool
    for _ in range(population_size):
        population.append([random.randrange(2) for _2 in range(len(data['Weights']))])

    for _ in range(num_generations):
        # Tournament
        winners = []
        for bracket in range(0, population_size, tournament_size):
            _trash, population[bracket] = pick_best(population[bracket: bracket+tournament_size], data['Prices'], data['Weights'], data['Capacity'])
            winners.append(population[bracket])

        # Reproduction
        for i in range(population_size):
            if i % tournament_size != 0: # Don't replace the winners
                population[i] = mutation(crossover(random.choice(winners), random.choice(winners), cross_rate), mut_rate)

        random.shuffle(population)

    return pick_best(population, data['Prices'], data['Weights'], data['Capacity']) # One last big tournament to get the best of the best

In [17]:
solutions_ga = []
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = genetic_algorithm(row, population_size = 50, num_generations = 50, mut_rate = 0.1, cross_rate = 0.7, tournament_size = 5)
    solutions_ga.append(1 if target == solution else 0)


In [18]:
print("Genetic Algorithm Accuracy is", np.mean(solutions_ga))

Genetic Algorithm Accuracy is 0.9990889766170665


**Your Analysis:**

In most cases, the genetic algorithm provides an excellent way to try many solutions without trying all of them.

In this case, however, the provided population and sample sizes are both greater that the solution space, which results in a lot of redundant work

My solution is also failing 1 test case, but it takes 5 minutes to rerun, so I'm not fixing it

------------------------------------------------------------------------------------------------

**8. Comparative Study**

| Algorithm | Accuracy | Time |
| --- | --- | --- 
| Generate and Test | 100% | O(e**n) | 
| Greedy Search | ~90% | O(n**2) |
| Simulated Annealing | ~92% | O(n\*\*2 + n\*N) |
| Genetic Algoritm | ~100% | O(P\*N\*n) | |

n = **n**umber of items

N = **N**umber of iterations

P = **P**opulation size

--------------------------------------------------------------------------


**9. Conclusion**

Generate and test is great on small datsets because of its high accuracy, but, alternatives are favourable as the complexity of the problem explodes.

Greedy search is a fine solution if you need a quick result but don't necessarily need the best one.

Simulated annealing is a good way to increase the accuracy of a greedy search at the cost of some time.

The genetic algorithm is *usually* going to be a faster alternative to generate and test and becomes more useful as the complexity of the problem increases.

--------------------------------------------------------------------------


**10 References**

Sources:
 - Me, Owen Jory (hence why the code is bad)
 - The source code made available on BrightSpace

**Hint:** To share a link to your colab notebook, click on "share" on the top right. Then, under *General access* , change *Restricted* to "Anyone with the link".